In [6]:
from openai import OpenAI
from tqdm import trange
import sudokum
import numpy as np
import re
import os

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

def llm(prompt, verbose=False):
    if verbose: print(f"> {prompt}")
    completion = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user",
                   "content": prompt 
                   }],
    )
    answer = completion.choices[0].message.content
    if verbose: print(f"< {answer}")
    return answer


def format_sudoku(puzzle):
    return str(np.array(puzzle))

# def parse_sudoku(text):
#     puzzle = [[]]
#     for number in re.findall(r'\d', text):
#         if len(puzzle[-1]) == 9:
#             puzzle.append([])
#         puzzle[-1].append(int(number))
#     return np.array(puzzle)
        
def parse_sudoku(text):
    puzzle = []
    numbers = re.findall(r'\d', text)
    for i in range(9):
        row = numbers[i*9:(i+1)*9]
        # Pad the row with zeros if it's incomplete
        row += [0] * (9 - len(row))
        puzzle.append(row)
    return np.array(puzzle, dtype=int)

client = OpenAI(api_key=OPENAI_API_KEY)
def run_llm(puzzle, prompt, few_shot=0):
    for _ in range(few_shot):
        p = sudokum.generate(mask_rate=0.1)
        _, s = sudokum.solve(p)
        p, s= np.array(p), np.array(s)
        prompt += "Here is an example:\n"
        prompt += format_sudoku(p)
        prompt += "The solution is:\n"
        prompt += format_sudoku(s)
        prompt += "\n\n"

    prompt += "The sudoku is:" + "\n\n" + format_sudoku(puzzle)
    print("=================================================")
    print(prompt)
    
    answer = llm(prompt)
#    answer = parse_sudoku(answer)
# NEW
    try:
        answer = parse_sudoku(answer)
        if answer.shape != (9, 9):
            print("Warning: Parsed answer is not a 9x9 grid. Returning original puzzle.")
            return puzzle
        print('\nSolution\n')
        print(answer)
    except Exception as e:
        print(f"Error parsing LLM answer: {e}")
        print("Returning original puzzle.")
        return puzzle
# END NEW
    return answer
    


N = 25
cnt_correct = 0
cnt = 0
accuracy = 0
iterator = trange(N)

prompt = """Solve this 9x9 sudoku by replacing 0s with numbers 1 to 9.
            Only print the solution, no other text.\n\n"""

metaprompt = f"Refine the following instructions for a LLM solving a sudoku puzzle. Make sure to make it better at solving the task. ```{prompt}```"

prompt = llm(metaprompt, verbose=True)

for line in iterator:
    puzzle = sudokum.generate(mask_rate=0.3)
    _, solution = sudokum.solve(puzzle)
    puzzle, solution = np.array(puzzle), np.array(solution)
    
    answer = run_llm(puzzle, prompt=prompt, few_shot=0)
    correct = np.all(answer == solution)
    cnt_correct += correct
    cnt += 1
    accuracy = cnt_correct / cnt
    iterator.set_description(f"Accuracy: {accuracy:.3f} ({cnt_correct}/{cnt})")


  0%|          | 0/25 [00:00<?, ?it/s]

> Refine the following instructions for a LLM solving a sudoku puzzle. Make sure to make it better at solving the task. ```Solve this 9x9 sudoku by replacing 0s with numbers 1 to 9.
            Only print the solution, no other text.

```
< Instructions for LLM:

Your task is to solve a given sudoku puzzle that is of the size 9x9. The puzzle will contain numbers from 1 to 9 and some cells may contain 0, signifying that they are empty and need to be filled. Follow the rules below to solve the puzzle:

1. Each row must contain each number from 1 to 9 exactly once.
2. Each column must also contain each number from 1 to 9 exactly once.
3. Every 3x3 sub-grid within the larger 9x9 grid should contain each number from 1 to 9 exactly once.

You should use well established algorithms or techniques to solve the sudoku, such as elimination of possibilities, cross-referencing, and looking ahead.

Please output the solution only, without any additional comments, instructions, or step-by-step explan

Accuracy: 0.000 (0/1):   4%|▍         | 1/25 [00:12<05:00, 12.51s/it]


Solution

[[1 8 7 4 3 9 2 6 5]
 [2 5 4 8 7 6 1 3 9]
 [9 6 3 2 1 5 8 7 4]
 [4 7 9 6 5 8 3 2 1]
 [8 2 1 3 9 4 6 5 7]
 [6 3 5 1 7 2 4 9 8]
 [5 1 6 7 4 2 9 8 3]
 [7 4 8 9 6 3 5 1 2]
 [3 9 2 5 8 1 7 4 6]]
Instructions for LLM:

Your task is to solve a given sudoku puzzle that is of the size 9x9. The puzzle will contain numbers from 1 to 9 and some cells may contain 0, signifying that they are empty and need to be filled. Follow the rules below to solve the puzzle:

1. Each row must contain each number from 1 to 9 exactly once.
2. Each column must also contain each number from 1 to 9 exactly once.
3. Every 3x3 sub-grid within the larger 9x9 grid should contain each number from 1 to 9 exactly once.

You should use well established algorithms or techniques to solve the sudoku, such as elimination of possibilities, cross-referencing, and looking ahead.

Please output the solution only, without any additional comments, instructions, or step-by-step explanations. The solution should be formatted

Accuracy: 0.000 (0/2):   8%|▊         | 2/25 [00:17<03:10,  8.28s/it]

...


Solution

[[5 6 8 2 3 9 1 7 4]
 [1 9 3 7 4 6 8 2 5]
 [4 7 2 1 8 5 3 9 6]
 [6 4 5 3 2 8 9 1 7]
 [3 8 7 6 9 1 4 5 2]
 [2 1 9 5 7 4 6 8 3]
 [7 2 1 8 6 3 5 4 9]
 [9 5 6 4 1 2 7 3 8]
 [8 3 4 9 5 7 2 6 1]]
Instructions for LLM:

Your task is to solve a given sudoku puzzle that is of the size 9x9. The puzzle will contain numbers from 1 to 9 and some cells may contain 0, signifying that they are empty and need to be filled. Follow the rules below to solve the puzzle:

1. Each row must contain each number from 1 to 9 exactly once.
2. Each column must also contain each number from 1 to 9 exactly once.
3. Every 3x3 sub-grid within the larger 9x9 grid should contain each number from 1 to 9 exactly once.

You should use well established algorithms or techniques to solve the sudoku, such as elimination of possibilities, cross-referencing, and looking ahead.

Please output the solution only, without any additional comments, instructions, or step-by-step explanations. The solution should be formatted

Accuracy: 0.417 (10/24):  96%|█████████▌| 24/25 [02:05<00:04,  4.84s/it]


Solution

[[4 5 6 7 1 8 3 2 9]
 [8 3 4 6 5 2 9 1 7]
 [2 1 7 3 9 4 6 8 5]
 [9 4 3 8 7 5 2 6 1]
 [5 6 8 9 3 1 4 7 2]
 [1 7 2 4 2 6 5 3 9]
 [7 4 5 2 8 3 1 9 6]
 [6 2 9 1 4 7 8 5 3]
 [3 8 1 5 6 9 7 4 2]]
Instructions for LLM:

Your task is to solve a given sudoku puzzle that is of the size 9x9. The puzzle will contain numbers from 1 to 9 and some cells may contain 0, signifying that they are empty and need to be filled. Follow the rules below to solve the puzzle:

1. Each row must contain each number from 1 to 9 exactly once.
2. Each column must also contain each number from 1 to 9 exactly once.
3. Every 3x3 sub-grid within the larger 9x9 grid should contain each number from 1 to 9 exactly once.

You should use well established algorithms or techniques to solve the sudoku, such as elimination of possibilities, cross-referencing, and looking ahead.

Please output the solution only, without any additional comments, instructions, or step-by-step explanations. The solution should be formatted

Accuracy: 0.400 (10/25): 100%|██████████| 25/25 [02:11<00:00,  5.26s/it]


Solution

[[5 9 3 6 7 8 2 1 4]
 [1 2 6 9 3 5 4 8 7]
 [4 8 7 1 2 4 6 9 3]
 [7 3 4 8 9 2 5 6 1]
 [2 5 8 4 6 1 3 7 9]
 [6 1 9 7 5 3 8 2 4]
 [8 4 5 2 1 7 9 3 6]
 [9 7 2 3 4 6 1 5 8]
 [3 6 1 5 8 9 7 4 2]]
